# Análisis de ED con SymPy

## Introducción

In [261]:
%pip install sympy

Note: you may need to restart the kernel to use updated packages.


In [262]:
import sympy
from sympy import *

Sympy define el objeto Symbols que resprenta el nombre de una variable simbólica, de una función, u otras expresiones matemáticas (significa que es un símbolo, no una letra por ej.)
  Toma como argumento un string y devuelve un objeto de tipo simbolo. Y entiende que es un símbolo no un valor numérico. 

In [263]:
t = symbols("t")

In [264]:
type(t)

sympy.core.symbol.Symbol

In [265]:
suma = t + 1
suma.subs(
    t, 2
)  # sustituir el valor de t en la ecuación suma, con el valor dado por argumento

3

Para representar las funciones usamos Funcion() 

In [266]:
f = Function("f")
type(f)

sympy.core.function.UndefinedFunction

In [267]:
f(t)

f(t)

## Ecuaciones diferenciales en SymPy


diff() representa la derivada de la función que se le pasa por argumento. En este caso, la primera derivada. 

In [268]:
dfdt = diff(
    f(t), t
)  # dfdt representa la derivada de f con respecto al tiempo. No la está calculando aún.
dfdt

Derivative(f(t), t)

Para representar una ED utilizamos Eq()

In [269]:
alpha = symbols("alpha")
eq1 = Eq(dfdt, alpha * f(t))  # Representamos ambos miembros de la ecuación
eq1

Eq(Derivative(f(t), t), alpha*f(t))

Para resolver la ecuación utilizamos dsolve()

In [270]:
solution_eq = dsolve(eq1)
solution_eq

Eq(f(t), C1*exp(alpha*t))

Esta es la solucion general de la ED, contiene un parámetro que no está especificado su valor.\
Para obtener una solución particular de la ecuación, consideramos las condiciones de contorno.\
Buscamos la solución para t = 0; nos dará la p0 población en t inicial. 

In [271]:
C1, p_0 = symbols("C1 p_0")
particular = solution_eq.subs(C1, p_0)
particular

Eq(f(t), p_0*exp(alpha*t))

Expresión de creciemiento exponencial de población, dependiendo del exponente de e es si la población es creciente o decreciente. [Para mas info.](https://es.wikipedia.org/wiki/Crecimiento_exponencial)

##Ecuación cuadrática

 1- Definimos los símbolos: r, k t y x

In [272]:
r, K, t = sympy.symbols("r K t")

Definir a *x* como una función

In [273]:
x = sympy.Function("x")

In [274]:
# Definimos el diferencial
dxdt = sympy.diff(x(t), t)
dxdt

Derivative(x(t), t)

In [275]:
# Definimos el lado derecho de la ecuación
lado_derecho = r * x(t) * (1 - x(t) / K)

In [276]:
# Definimos la ecuación completa (lado der + lado izq)
# ED Continua, a la que llegamos 3.7
ecuacion = sympy.Eq(dxdt, lado_derecho)

In [277]:
ecuacion

Eq(Derivative(x(t), t), r*(1 - x(t)/K)*x(t))

2-  Solución general: consiste en resolver la ED.

In [278]:
solucion_gral = sympy.dsolve(ecuacion)

In [279]:
solucion_gral

Eq(x(t), K*exp(C1*K + r*t)/(exp(C1*K + r*t) - 1))

In [280]:
general_d = solucion_gral.rhs  # ? return the right-hand side of the relation.

3- Aplicamos las condiciones de contorno. Solución para t = 0, encontramos la población inicial Xo. Consiste en reemplazar t por 0. 

In [281]:
solucion_t0 = general_d.subs(t, 0)  # sería la población inicial

In [282]:
solucion_t0  # Solución particular para t = 0

K*exp(C1*K)/(exp(C1*K) - 1)

4- Encontramos el valor de C1 para t = 0. Es el valor de C1 para la población incial. Consiste en despejar C1 de la solución con condición de contorno t = 0. Nos queda expresada C1 en función de la población inicial.

 Crea symbols para p_t0 y C1

In [283]:
C1, x_0 = sympy.symbols("C1 x_0")  # x_0 población inicial

In [284]:
ec_temporal = sympy.Eq(
    solucion_t0, x_0
)  # arma la ecuación con la solución particular para t = 0 y su resultado que es la población incial

In [285]:
ec_temporal

Eq(K*exp(C1*K)/(exp(C1*K) - 1), x_0)

In [286]:
solucion = sympy.solve(
    ec_temporal, C1
)  # Resuelve la ecuación algebraica en función de C1.

In [287]:
solucion  # solucion particular de C1 para t = 0.

[log(-x_0/(K - x_0))/K]

Devuelve la solución en forma de lista, ya que en algunas ocasiones existe más de una solucion para la ecuación.De esta manera, cada elemento de la lista será una solución.

In [288]:
valor_c1 = solucion[0]

In [289]:
valor_c1

log(-x_0/(K - x_0))/K

5- A partir de la solución gral, sustituimos C1 por el valor que se encontró valor_c1. Valor expresado en población inicial. 

In [290]:
general_d

K*exp(C1*K + r*t)/(exp(C1*K + r*t) - 1)

In [291]:
particular = general_d.subs(C1, valor_c1)

In [292]:
particular  # solución particular en función a la constante en la población inical

-K*x_0*exp(r*t)/((K - x_0)*(-x_0*exp(r*t)/(K - x_0) - 1))

In [293]:
particular_simplificada = sympy.simplify(particular)
particular_simplificada

K*x_0*exp(r*t)/(K + x_0*exp(r*t) - x_0)

Esta función se llama *Ecuación logística* y se utiliza para modelos de crecimiento

## Ecuación logística 

Vamos a comprobar que la ecuación logistica que encontramos (particular_simplificada) es equivalente a la de la bibliografía ([ver acá](https://es.wikipedia.org/wiki/Funci%C3%B3n_log%C3%ADstica)), las vamos a restar y debería dar cero.

$$f(t) = \frac{K}{1 + A \exp(-rt)}$$ 

donde $A = (K - p_0) / p_0$.

In [294]:
A = (K - x_0) / x_0

In [295]:
ec_log = K / (1 + A * sympy.exp(-r * t))

In [296]:
ec_log

K/(1 + (K - x_0)*exp(-r*t)/x_0)

In [297]:
ec_log - particular_simplificada

-K*x_0*exp(r*t)/(K + x_0*exp(r*t) - x_0) + K/(1 + (K - x_0)*exp(-r*t)/x_0)

In [298]:
sympy.simplify(ec_log - particular_simplificada)

0

In [299]:
sympy.simplify(ec_log) - sympy.simplify(particular_simplificada)

0

## Sustituciones
https://en.wikipedia.org/wiki/Estimates_of_historical_world_population

In [300]:
sustituciones = [(t, 0), (alpha, 0.025), (x_0, 2.5), (K, 13.88)]

In [301]:
particular_simplificada.subs(
    sustituciones
)  # con tiempo inicio me devuelve la pblacion inicia

2.50000000000000

In [302]:
sustituciones = [(t, 2), (r, 0.025), (x_0, 2.5), (K, 13.88)]

In [303]:
particular_simplificada.subs(sustituciones)

2.60412936777885

In [304]:
sustituciones = [(t, 2.5), (r, 0.025), (x_0, 2.5), (K, 13.88)]

In [305]:
particular_simplificada.subs(sustituciones)

2.63067705271566

Comparar los resultados con las poblaciones de la wiki. Por ej. en
1990= 5.2 mil millones de habitantes.\
Cerca del 1984 debería dar muy similar, es para un valor de t = 34.
Con la ecuación continua puedo calcular valores entre los años.Por ej. podría calcular la población para el año 34,5

In [306]:
sustituciones = [(t, 40), (r, 0.025), (x_0, 2.5), (K, 13.88)]
particular_simplificada.subs(sustituciones)

5.18958586052619

In [307]:
sustituciones = [(t, 40), (r, 0.025), (x_0, 2.5), (K, 13.88)]
ec_log.subs(sustituciones)

5.18958586052619

## Ejercicio 1

 Use SymPy para resolver la ecuación de crecimiento cuadrático con otro tipo de parametrización (ecuación 3.8 del apunte):

$$ \frac{dx(t)}{dt} = \alpha x(t) + \beta x^2(t) $$
